In [ ]:
# !pip install mapclassify
# !pip install geopandas

In [ ]:
import zipfile
from zipfile import ZipFile

import matplotlib
%matplotlib inline

import geopandas as gpd
import mapclassify

import pandas as pd

# Learning objectives

1. Understand geospatial data in Python
2. Download map of USA and plot it; do the same thing with States and California counties
3. Learn how to join population data with the spatial-geometric data

# Introduction to geospatial data in Python

"[GeoPandas](https://geopandas.org/) is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting."

![geo1](../../Img/Geo1.png)

![geo2](../../Img/Geo2.png)

![geo3](../../Img/Geo3.png)

![geo4](../../Img/Geo4.png)

![geo5](../../Img/Geo5.png)

![geo6](../../Img/Geo6.png)

![geo7](../../Img/Geo7.png)

![geo8](../../Img/Geo8.png)

![geo9](../../Img/Geo9.png)

![geo10](../../Img/Geo10.png)

![geo11](../../Img/Geo11.png)

![geo12](../../Img/Geo12.png)

# DIVA (Data-Interpolating Variational Analysis)

[DIVA-GIS](https://www.diva-gis.org/gdata) is an excellent website for downloading free geographic (GIS) data for any country in the world. The [DIVA-GIS software manual](https://www.diva-gis.org/docs/DIVA-GIS_manual_7.pdf) is worth a read as well. 

### USA example
1. Create a folder named USA (or whatever country/state/region you are working with) in your DIGHUM101-2019 directory (this has already been done for you)
2. Visit [DIVA-GIS](https://www.diva-gis.org/gdata) and select "United States" from the "Country" dropdown menu
3. Select "Administrative Areas" from the "Subject" dropdown menu
4. Click "OK"
5. Click "Download"

### Unzip the file!

In [ ]:
# Define a variable for the ZipFile function that contains the path to the zip file 
# NOTE: you may have to change your file paths to match
zf = ZipFile("/Users/evan.admin/Desktop/DIGHUM101-2020/Data/Geo/USA_adm.zip", "r")

# Use the extractall method with the file path where you want the extracted files to go
zf.extractall("/Users/evan.admin/Desktop/DIGHUM101-2020/Notebooks/Week3/USA")

# Stop the process
zf.close()

### Define our shapefile

Shapefiles contain the geometric data!

In [ ]:
usa = gpd.read_file("/Users/evan.admin/Desktop/DIGHUM101-2020/Notebooks/Week3/USA/USA_adm0.shp")
print(type(usa))
usa

In [ ]:
# Plot it!
usa.plot()

# State boundaries

Now we need to get the state boundaries to overlay on this map. Visit this URL to download them: https://raw.githubusercontent.com/dlab-geo/geopandas_intro/master/data/us_states.zip

In [ ]:
# Read the file
# Note this other zip convention
state_boundaries = gpd.read_file("zip:///Users/evan.admin/Desktop/DIGHUM101-2020/Data/Geo/us_states.zip")
state_boundaries.head()

In [ ]:
# Plot the state boundaries
state_boundaries.plot(
    # Width of boundary line
    linewidth=0.25, 
    
    # Boundary color line
    edgecolor='white', 
    
    # State color
    facecolor='green',
    
    # Figure size
    figsize=(14,10));

In [ ]:
# Spatial subset of the contiguous US (zoom in!)
state_boundaries.cx[-130:-70,25:50].plot(linewidth=0.25, 
                                         edgecolor='white', 
                                         facecolor='black', 
                                         figsize=(14,10))

# Individual county polygons

To get individual state county boundaries, visit https://www.census.gov/geographies/mapping-files/2018/geo/carto-boundary-file.html

We want the file "cb_2018_us_county_5m.zip" What is California's STATEFP code? 

In [ ]:
counties = gpd.read_file("zip:///Users/evan.admin/Desktop/DIGHUM101-2020/Data/Geo/cb_2018_us_county_5m.zip")
counties.head()

In [ ]:
# Just california...
cal_counties = counties[counties["STATEFP"] == "06"]
cal_counties.head()

In [ ]:
# Pull the state info for California
state_boundaries.head()

In [ ]:
california = state_boundaries[state_boundaries["ABBREV"] == "CA"]

In [ ]:
# Plot it
california.plot(); 

Visi the [Census Bureau website](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html)to get information about these counties. 

In [ ]:
pop = pd.read_csv("/Users/evan.admin/Desktop/DIGHUM101-2020/Data/Geo/DEC_10_SF1_GCTPH1.ST05_with_ann.csv")
pop.head()

In [ ]:
# Add a zero to the new GEOID column
pop["GEOID"] = ["0" + str(x) for x in pop["Target Geo Id2"]]
pop.head()

# Do the join!

Now we can combine `cal_counties` and `pop` because they have the same number of rows. 

In [ ]:
pop.shape

In [ ]:
cal_counties.shape

In [ ]:
# cal_counties
# pop
geo_pop = cal_counties.merge(pop, on = "GEOID", how = "left")
geo_pop.head()

# Plot

In [ ]:
geo_pop.plot(
    # Which column to visualize?
    column = "Population", 
    
    # Which color palette to use?
    cmap = "Reds", 
    
    # What color should be used to separate the counties?
    edgecolor = "black", 
    
    # Width of the county boudary lines
    linewidth = 0.1, 
    
    # Change figure size
    figsize = (8, 8),
    
    # Add legend
    legend = True,
    
    # What do the different classifier schemes do? 
    scheme = "quantiles");

Visit the [this page](https://pysal.org/mapclassify/_modules/mapclassify/classifiers.html) and pass in different schemes. How does `scheme = "Equal_Interval"` differ from "quantiles"? 

> HINT: This is how people [lie with maps](https://www.amazon.com/How-Lie-Maps-Mark-Monmonier/dp/0226534219)!

[Learn more here](https://github.com/pysal/mapclassify).

In [ ]:
geo_pop.plot(column = "Population", 
             cmap = "Greens", 
             edgecolor = "black", 
             linewidth = 0.1, 
             scheme = "Equal_Interval", 
             legend = True, 
             figsize = (8,8));

# Going further

See if you can get the Geospatial Fundamentals in Python workshop materials to work!

https://github.com/dlab-berkeley/Geospatial-Fundamentals-in-Python